In [48]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import Polygon, mapping
from rasterio.features import geometry_mask
import numpy as np

import os
from datetime import datetime

In [49]:
file_path = '/home/kea-trainee-punpun/task1/data/CPM_lstm_2023.tiff'

In [50]:
def latlon_to_pixel(lat, lon, transform):
    col, row = ~transform * (lon, lat)
    return col, row

In [51]:
polygon_coords = [
    (101.5, 15.5),
    (100.0, 15.5),
    (99.0, 16.0),
    (101.5, 15.0),
    (104, 14)
]

In [52]:
def crop_raster_with_polygon(tiff_file, polygon_coords):
    # Open the TIFF file
    with rasterio.open(tiff_file) as src:
        # Create a Shapely polygon from the coordinates
        polygon = Polygon(polygon_coords)

        geojson_polygon = mapping(polygon)
        
        # Perform the rasterio mask
        out_image, out_transform = mask(src, [polygon], crop=True)
        out_meta = src.meta.copy()
        
        return out_meta, out_image, out_transform

In [53]:
out_meta, out_image, out_transform = crop_raster_with_polygon(file_path, polygon_coords)

In [54]:

out_meta.update({
    'height': out_image.shape[1],
    'width': out_image.shape[2],
    'transform': out_transform
})

In [55]:
def generate_output_file(out_meta):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"output_{timestamp}.tiff"
    with rasterio.open(f'/home/kea-trainee-punpun/Week2/polygon_tiff_output/{filename}', 'w', **out_meta) as dst:
        dst.write(out_image)
    print(f"File '{filename}' created.")

In [56]:
generate_output_file(out_meta)

File 'output_20240619_083833.tiff' created.
